# General configuration

## Libraries

In [ ]:
import os
import sys
import importlib
from abc import abstractmethod
from typing import Tuple, Any

import numpy as np
import torch
import tensorflow as tf
import matplotlib.pyplot as plt

## Random seeds

Set random seeds for reproducibility

In [2]:
np.random.seed(0)
torch.manual_seed(0)
tf.random.set_seed(0)

## Take care of paths

In [ ]:
print(f"Current working directory :{os.getcwd()}")
print(f"Current sys path :{sys.path}")

In [4]:
expected_path="/mnt/c/Users/edanbaz/OneDrive - Ericsson/Documents/projects/illia"
if not os.path.exists(expected_path):
    raise Exception("Please ensure correct path")

In [ ]:
os.chdir(os.path.join(expected_path,'examples'))
print(f"Changed working directory to :{os.getcwd()}")

if expected_path not in sys.path:
    sys.path.insert(0, expected_path)
    print("Added {expected_path} to sys.path")

print(f"Updated sys.path : {sys.path}")

# Test - Load illia and show available backends 

When the backend is selected we can import illia, if we want to change the backend we need to restart the kernel. The backend can't be changed dynamically.

In [6]:
import illia

We can check the available backends using the following function:

In [ ]:
illia.show_available_backends()

# Test - Distributions

## Test params and utils

In [8]:
shape = (3, 2)
mu_prior = 0.0
std_prior = 0.1
mu_init = 0.0
rho_init = -7.0

In [9]:
def compare_tensors(a, b, rtol=1e-1, atol=1e-1, name=""):
    are_close = np.allclose(a, b, rtol=rtol, atol=atol)
    max_diff = np.max(np.abs(a - b))
    print(f"{name} are close: {are_close}")
    print(f"Max absolute difference for {name}: {max_diff}")
    return are_close

## Initialize class distributions

In [10]:
from illia.torch.distributions.dynamic.gaussian import GaussianDistribution as TorchGaussianDistribution
from illia.tf.distributions.dynamic.gaussian import GaussianDistribution  as TFGaussianDistribution

In [11]:
torch_dynamic_dist = TorchGaussianDistribution(
    shape=shape, mu_init=mu_init, rho_init=rho_init
)

In [12]:
tf_dynamic_dist = TFGaussianDistribution(
    shape=shape, mu_init=mu_init, rho_init=rho_init
)

## Test 1 - Distributions sampling

In [ ]:
print("Test 1: Sampling")

n_samples = 10000
torch_samples = np.array([torch_dynamic_dist.sample().detach().cpu().numpy() for _ in range(n_samples)])
tf_samples = np.array([tf_dynamic_dist.sample().numpy() for _ in range(n_samples)])

Compare means

In [ ]:
torch_mean = np.mean(torch_samples, axis=0)
tf_mean = np.mean(tf_samples, axis=0)
compare_tensors(torch_mean, tf_mean, name="Means")

Compare standard deviations

In [ ]:
torch_std = np.std(torch_samples, axis=0)
tf_std = np.std(tf_samples, axis=0)
compare_tensors(torch_std, tf_std, name="Standard deviations")

## Test 2 - Distributions log probs

In [ ]:
print("\nTest 2: Log probability")

x = np.random.randn(*shape).astype(np.float32)
torch_log_prob = torch_dynamic_dist.log_prob(torch.tensor(x, dtype=torch.float32)).detach().cpu().numpy()
tf_log_prob = tf_dynamic_dist.log_prob(tf.constant(x, dtype=tf.float32)).numpy()
compare_tensors(torch_log_prob, tf_log_prob, rtol=1e-1, atol=1e-1, name="Log probabilities")

## Test 3 - Distributions num of params

In [ ]:
# Test 3: Number of parameters
print("\nTest 3: Number of parameters")
print("PyTorch num params:", torch_dynamic_dist.num_params)
print("TensorFlow num params:", tf_dynamic_dist.num_params)
print("Num params are equal:", torch_dynamic_dist.num_params == tf_dynamic_dist.num_params)

## Prove visually as well if implementations are correct or not

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(121)
plt.hist(torch_samples.flatten(), bins=50, alpha=0.5, label='PyTorch')
plt.hist(tf_samples.flatten(), bins=50, alpha=0.5, label='TensorFlow')
plt.legend()
plt.xlabel("values")
plt.ylabel("frequency")
plt.title("Distribution of Samples")

plt.subplot(122)
plt.scatter(torch_samples.flatten()[:1000], tf_samples.flatten()[:1000], alpha=0.1)
plt.plot([-3, 3], [-3, 3], 'r--')
plt.xlabel("PyTorch Samples")
plt.ylabel("TensorFlow Samples")
plt.title("PyTorch vs TensorFlow Samples")

plt.tight_layout()
plt.show()

# Test - Bayesian Module (nn.base)

In [19]:
from illia.torch.nn.base import BayesianModule as TorchBayesianModule
from illia.tf.nn.base import BayesianModule as TFBayesianModule

## Class definitions - forward method to test

In [20]:
class TorchTestModule(TorchBayesianModule):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(10, 5)

    def forward(self, x):
        return self.linear(x)

    def kl_cost(self):
        return torch.tensor(1.0), 1

In [21]:
class TFTestModule(TFBayesianModule):
    def __init__(self):
        super().__init__()
        self.linear = tf.keras.layers.Dense(5)

    def call(self, x):
        return self.linear(x)

    def kl_cost(self):
        return tf.constant(1.0), 1

## Initialize modules

In [22]:
torch_module=TorchTestModule()

In [23]:
tf_module=TFTestModule()

## Test 1 - Bayesion Module freeze and unfreeze

In [24]:
def test_freeze_unfreeze():
    
    print("Testing freeze and unfreeze...")
    
    # Test PyTorch module
    assert not torch_module.frozen, "PyTorch module should not be frozen initially"
    torch_module.freeze()

    assert torch_module.frozen, "PyTorch module should be frozen after freeze()"
    torch_module.unfreeze()

    assert not torch_module.frozen, "PyTorch module should not be frozen after unfreeze()"

    # Test TensorFlow module
    assert not tf_module.frozen, "TensorFlow module should not be frozen initially"
    tf_module.freeze()

    assert tf_module.frozen, "TensorFlow module should be frozen after freeze()"
    tf_module.unfreeze()

    assert not tf_module.frozen, "TensorFlow module should not be frozen after unfreeze()"
    
    print("Freeze and unfreeze test passed!",'\n\n')

## Test 2 - Bayesion Module KL cost

In [25]:
def test_kl_cost():

    print("Testing KL cost...")
    
    torch_kl, torch_n = torch_module.kl_cost()
    tf_kl, tf_n = tf_module.kl_cost()

    print(f'\nPyTorch : {torch_kl.item()}, {torch_n}')
    print(f'TensorFlow : {tf_kl.numpy()}, {tf_n}\n')

    assert torch_kl.item() == tf_kl.numpy(), f"KL divergence mismatch: PyTorch {torch_kl.item()}, TensorFlow {tf_kl.numpy()}"
    assert torch_n == tf_n, f"N mismatch: PyTorch {torch_n}, TensorFlow {tf_n}"
    
    print("KL cost test passed!",'\n\n')

## Test 3 - Bayesion Module forward pass

In [26]:
def test_forward_pass():

    print("Testing forward pass...")
    
    # Input data
    input_data = np.random.randn(1, 10).astype(np.float32)
    
    # PyTorch forward pass
    torch_input = torch.from_numpy(input_data)
    torch_output = torch_module(torch_input)

    # TensorFlow forward pass
    tf_input = tf.convert_to_tensor(input_data)
    tf_output = tf_module(tf_input)

    # Compare outputs
    torch_np = torch_output.detach().numpy()
    tf_np = tf_output.numpy()
    
    max_diff = np.max(np.abs(torch_np - tf_np))
    print(f"Maximum absolute difference: {max_diff}")
    
    if max_diff > 1e-1:
        print("""
              Warning-Ignore for now: Outputs differ slighlty,this might be due to different 
              initialization or computational differences between PyTorch and TensorFlow for 
              torch.nn.Linear && tf.keras.layers.Dense 
              """
        )
        print("PyTorch output:", torch_np)
        print("TensorFlow output:", tf_np)
    else:
        print("Outputs are close enough.")

    # Use a more lenient comparison
    np.testing.assert_allclose(torch_np, tf_np, rtol=1, atol=1)
    
    print("Forward pass test passed!",'\n\n')

## Run all tests

In [ ]:
def run_all_tests():
    
    test_freeze_unfreeze()
    test_kl_cost()
    test_forward_pass()

run_all_tests()

# Test - Bayesian Module (nn.losses)

## Import specific classes for each backend

In [28]:
from illia.torch.nn.losses import (
    KLDivergenceLoss as TorchKLDivergenceLoss, 
    ELBOLoss as TorchELBOLoss 
)
from illia.torch.nn.linear import Linear as TorchLinear

from illia.tf.nn.losses import (
    KLDivergenceLoss as TFKLDivergenceLoss, 
    ELBOLoss as TFELBOLoss
)
from illia.tf.nn.linear import Linear as TFLinear

## Using the classes for each backend defined previously

In [29]:
class TorchTestModule(TorchBayesianModule):

    def __init__(self):

        super().__init__()

        self.linear = TorchLinear(10, 5)

    def forward(self, x):

        return self.linear(x)

    def kl_cost(self):

        return torch.tensor(1.0), 1

In [30]:
class TFTestModule(TFBayesianModule):

    def __init__(self):

        super().__init__()

        self.linear = TFLinear(10, 5)

    def call(self, x):

        return self.linear(x)

    def kl_cost(self):

        return tf.constant(1.0), 1

## Defining the classes

In [31]:
torch_kl_divengence = TorchKLDivergenceLoss()
torch_elbo_loss = TorchELBOLoss(loss_function=torch.nn.MSELoss())
torch_module = TorchTestModule()

In [32]:
tf_kl_divengence = TFKLDivergenceLoss()
tf_elbo_loss = TFELBOLoss(loss_function=tf.keras.losses.MeanSquaredError())
tf_module = TFTestModule()

## Test 1 - Check if parameters are available

In [33]:
def check_parameters():

    print("Check the existence of trainable parameters in the classes...")

    torch_list_parameters = list(torch_module.parameters())
    assert len(torch_list_parameters) != 0, "No parameters availables in TorchTestModule"

    trainable_variables = tf_module.trainable_variables
    assert len(trainable_variables) != 0, "No trainable parameters available in TFTestModule"
    
    print("Test passed!",'\n\n')

## Test 2 - Check losses

In [34]:
def check_forward_losses():

    print("Check the forward propagation of the loss functions...")
        
    # Input data
    input_data = np.random.randn(1, 10).astype(np.float32)
    y_true = np.random.randn(1, 10).astype(np.float32)
    y_pred = np.random.randn(1, 10).astype(np.float32)

    # PyTorch forward pass
    torch_input = torch.from_numpy(input_data)
    torch_output = torch_module(torch_input)
    torch_kl_divengence_output = torch_kl_divengence(torch_module)
    torch_elbo_loss_output = torch_elbo_loss(torch.from_numpy(y_true), torch.from_numpy(y_pred), torch_module)

    # Tensorflow forward pass
    tf_input = tf.convert_to_tensor(input_data)
    tf_output = tf_module(tf_input)
    tf_kl_divergence_output = tf_kl_divengence(tf_module)
    tf_elbo_loss_output = tf_elbo_loss(tf.convert_to_tensor(y_true), tf.convert_to_tensor(y_pred), tf_module)

    # Assert that the outputs are similar
    print("Torch output:", torch_output)
    print("TensorFlow output:", tf_output)
    print("Outputs are similar:", np.allclose(torch_output.detach().numpy(), tf_output))

    print("Torch KL divergence output:", torch_kl_divengence_output)
    print("TensorFlow KL divergence output:", tf_kl_divergence_output)
    print("KL divergence outputs are similar:", np.allclose(torch_kl_divengence_output.detach().numpy(), tf_kl_divergence_output))

    print("Torch ELBO loss output:", torch_elbo_loss_output)
    print("TensorFlow ELBO loss output:", tf_elbo_loss_output)
    print("ELBO loss outputs are similar:", np.allclose(torch_elbo_loss_output.detach().numpy(), tf_elbo_loss_output))

    print("Test passed!",'\n\n')

## Run all tests

In [ ]:
def run_all_tests():
    
    check_parameters()
    check_forward_losses()

run_all_tests()